In [0]:
import numpy as np
import pandas as pd
import zipfile
import json
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
pip install kaggle

In [10]:
!ls -a

.  ..  .config	.kaggle  sample_data


In [11]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"envykhamidov","key":"fdd2921f97a333303ae41722b3421d9f"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)


In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [15]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [16]:
!kaggle competitions download -c recsys-iad-challenge

 82% 9.00M/10.9M [00:00<00:00, 16.4MB/s]
100% 10.9M/10.9M [00:00<00:00, 18.6MB/s]
 96% 225M/235M [00:02<00:00, 76.5MB/s]
100% 235M/235M [00:03<00:00, 80.2MB/s]
 99% 363M/368M [00:02<00:00, 88.4MB/s]
100% 368M/368M [00:02<00:00, 137MB/s] 


In [0]:
items_json = zipfile.ZipFile('/content/{/content}/competitions/recsys-iad-challenge/items.json.zip', 'r')
random_benchmark_csv = zipfile.ZipFile('/content/{/content}/competitions/recsys-iad-challenge/random_benchmark.csv.zip', 'r')
train_json = zipfile.ZipFile('/content/{/content}/competitions/recsys-iad-challenge/train.json.zip', 'r')


In [0]:
items_json.extractall()
random_benchmark_csv.extractall()
train_json.extractall()


In [19]:
!wget https://www.dropbox.com/s/5jkp5uq3vr7ynse/normalized_text.csv?dl=0

--2020-03-04 19:32:38--  https://www.dropbox.com/s/5jkp5uq3vr7ynse/normalized_text.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5jkp5uq3vr7ynse/normalized_text.csv [following]
--2020-03-04 19:32:38--  https://www.dropbox.com/s/raw/5jkp5uq3vr7ynse/normalized_text.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1c891d8df9723429f43271107b.dl.dropboxusercontent.com/cd/0/inline/AzSKG6Do6kqM6_HqBWx-AaN8UxMxYMp-mUfkoD3GpO-T7YrQl1y8TjtDqmnff8rX6YgZZOrnKzG5y1SmKKF0nPpcmGFhhiSh3c1ILGmh_i4mxmHxXRbNAQT-4yDRZMA42x8/file# [following]
--2020-03-04 19:32:38--  https://uc1c891d8df9723429f43271107b.dl.dropboxusercontent.com/cd/0/inline/AzSKG6Do6kqM6_HqBWx-AaN8UxMxYMp-mUfkoD3GpO-T7YrQl1y8TjtDqmnff8rX6YgZZOrnKzG5y1SmKKF0nPpc

In [23]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 6.3MB/s 


In [33]:
import string
import pymorphy2

def text_normalizer(text):
    return " ".join([morph.normal_forms(word)[0] for word in text.translate(table).lower().split()])

def transformation(line):
    item = json.loads(line)
    item["title"] = text_normalizer(item["title"])
    item["content"] = text_normalizer(item["content"])
    return item

chars_to_replace=string.punctuation
table = str.maketrans(chars_to_replace, " "*len(chars_to_replace))
morph = pymorphy2.MorphAnalyzer()
items_list = []

with tqdm(open('./items.json')) as inf:
    with Pool(cpu_count()) as p:
        items_list = list(p.imap(transformation, inf))


In [0]:
items_df = pd.DataFrame(items_list).set_index('itemId')
items_df = items_df.drop(columns = ['image']) 

In [0]:
#файл получен в ноутбуке до этого и для более быстрого доступа загружается через dropbox
items_df = pd.read_csv("/content/normalized_text.csv?dl=0") #для локального использования
#лемматизация проходит очень долго ~2.5 часа
#предыдущие 3 ячейки дают такой же результат если их запустить

In [21]:
%%time

from tqdm.auto import tqdm

#train_list = []
user_ids = []
item_ids = []
values = []
s = 0
with tqdm(open('./train.json')) as inf:
  for line in inf:
    s+=1
    train = json.loads(line)
    for i in train['trainRatings'].keys():
      item_ids.append(i)
    for j in train['trainRatings'].values():
      if j ==1:
        values.append(j+0.5)
      else:
        values.append(j-1)
    for k in range(len(train['trainRatings'].keys())):
      user_ids.append(train['userId'])
  


CPU times: user 49 s, sys: 3.67 s, total: 52.7 s
Wall time: 52.5 s


In [41]:
import gc
del items_list
gc.collect()
#если лемматизируем в ноутбуке

NameError: ignored

In [22]:
%%time

from scipy.sparse import coo_matrix
coo = coo_matrix((values, (user_ids, item_ids)), shape=(s, len(items_df['title'])))

CPU times: user 28.2 s, sys: 1.05 s, total: 29.3 s
Wall time: 29.2 s


In [23]:
import gc
del user_ids
del item_ids
del values
gc.collect()

0

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorizer = TfidfVectorizer(lowercase=False, min_df = 100, max_df = 0.01)
tfidf_titles = vectorizer.fit_transform(items_df.title.astype('U'))
tfidf_titles

<328050x3372 sparse matrix of type '<class 'numpy.float64'>'
	with 1222133 stored elements in Compressed Sparse Row format>

In [26]:
vectorizer1 = TfidfVectorizer(lowercase=False, min_df = 100, max_df = 0.01)
tfidf_contents = vectorizer1.fit_transform(items_df.content.astype('U'))
tfidf_contents

<328050x23676 sparse matrix of type '<class 'numpy.float64'>'
	with 12591384 stored elements in Compressed Sparse Row format>

In [0]:
import scipy
from scipy.sparse import hstack
features = scipy.sparse.hstack([tfidf_contents, tfidf_titles], format="csr")

In [28]:
features

<328050x27048 sparse matrix of type '<class 'numpy.float64'>'
	with 13813517 stored elements in Compressed Sparse Row format>

In [29]:
!pip install lightfm

     |████████████████████████████████| 307kB 4.4MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707616 sha256=52bb30f439f4fe8790f9b5a9acddbe8cb34c4cef74dd93c0b0a88a981770ec85
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
import lightfm as fm
lfm = fm.LightFM(loss = 'logistic', no_components = 100)

In [35]:
lfm.fit_partial(coo, epochs = 10, verbose =1, num_threads = 4, item_features = features) 

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [0]:
df = pd.read_csv("/content/random_benchmark.csv") 

In [0]:
preds = lfm.predict(user_ids = df.userId.values, item_ids = df.itemId.values, item_features = features)

In [0]:
df['score'] = preds
temp1 = df.sort_values('score')[::-1]
temp1.to_csv("log.csv", columns =['userId', "itemId"], index = False)